<a href="https://colab.research.google.com/github/IIF0403/Thesis/blob/main/SimSiam_modell_u14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import files
from google.colab import output
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from copy import deepcopy
import random

In [39]:
##### DATA #######

### Function to load a UCR time_series dataset from my github ###

def load_dataset(Dataset):
  #Dataset: The name of the dataset to load. Example: "ECG5000"

  #github URL
  url_raw = 'https://raw.githubusercontent.com/IIF0403/Thesis/main/data/'
  url_train = url_raw + Dataset+'/'+Dataset+'_TRAIN'
  url_test = url_raw + Dataset+'/'+Dataset+'_TEST'

  #Loading the data
  data_train = pd.read_csv(url_train,header=None)
  data_test = pd.read_csv(url_test, header=None)
  data = pd.concat((data_train, data_test))

  #Want all datasets to have classes as integers starting from 0 
  Y = data.values[:,0]
  classes = len(np.unique(Y))
  Y_transformed = ( (Y-Y.min())/(Y.max()-Y.min()) )*(classes-1)
  data[data.columns[0]] = Y_transformed

  #Inserting the name of the dataset as a column (for later use, when several datasets will be combined)
  data.insert(loc=0, column = "Dataset", value=Dataset) 

  #Inserting the length of the time series T as a column (for later use)
  T = data.shape[1]-2 #The length of the time series
  data.insert(loc=1, column = "T", value = T) 

  return data, classes


### Function to make a big dataset out of several datasets loaded from github ###
def make_big_dataset(Datasets):
  #Datasets: A list of the name of the datasets to load. Example: ["ECG5000", "FordA", "FordB"]

  #Loading first dataset
  data, classes =  load_dataset(Datasets[0])
  used_classes = classes #keeping track of the class-labels already used

  #Loading each of the datasets in the list and combining them all togehter in a big dataset
  for i in range(1, len(Datasets)):
    #loading the i´th dataset
    Dataset = Datasets[i]
    dataset, classes= load_dataset(Dataset)

    #Need to change the class-labels such that all the class-labels of the different datasets differ from eachother
    labels = dataset.values[:,2]
    transformed_labels = labels + used_classes 
    dataset[dataset.columns[2]] = transformed_labels
    
    used_classes += classes #keeping track of the class-labels already used

    data = pd.concat((data, dataset)) #adding the new dataset to the big dataset
  
  return data, used_classes



### Dataset class ###
class Timeseries_Dataset(Dataset):
  def __init__(self, Datasets, Drive=False, Save=None, transform=None):
    #Datasets: a list with the name of the datasets, can be a list og several or one dataset. If it is several datasets, they will be made into one big dataset
    #Drive: True means loading the already saved dataset from google drive, false means loading data from github
    #Save: name of new dataset, if we want to save the new dataset to google drive
    #transform: a given transformation of the data

    ##Loading the data
    #if the data is saved in Google Drive, load the data from Drive
    if (Drive == True):
      Dataset = Datasets[0]
      print("Loading '",Dataset,"' from Google Drive")
      #drive.mount("/content/gdrive")
      #data = pd.read_csv('/content/gdrive/My Drive/Datasets/'+Dataset+'.csv')

      drive.mount('/content/drive')
      data = pd.read_csv('/content/drive/MyDrive/Datasets/'+Dataset+'.csv')

      classes = len(np.unique(data.values[:,2]))
    
    #else load data from github
    else:
      #If "Datasets" contains several datasets then make a big dataset 
      if (len(Datasets)>1):
        print("Loading and combining ", Datasets," from github")
        data, classes = make_big_dataset(Datasets)
      
      #If "Datasets" only contains one dataset, load the dataset from github
      else:
        Dataset = Datasets[0]
        print("Loading '",Dataset,"' from github")
        data, classes = load_dataset(Dataset)
    
    #Save new dataset to Google drive as 'name' if Save = 'name' and not None
    if (Save!=None): #Save the new dataset to google drive as Save
      print("Saving new dataset to google drive")
      #drive.mount("/content/gdrive")
      #data.to_csv('/content/gdrive/My Drive/Datasets/'+Save+'.csv', index=False)

      drive.mount('/content/drive')
      data.to_csv('/content/drive/MyDrive/Datasets/'+Save+'.csv', index=False)
    
    self.dataframe = data
    self.transform = transform
    self.classes = classes
    self.Datasets = Datasets

    
  #defining the len(Dataset) function
  def __len__(self): 
    return len(self.dataframe)

  #defining the _getitem_ function which creates samples, such that when Dataset[i] is called; the i´th sample is returned
  def __getitem__(self, key): 

    if isinstance(key, slice): # if given a slicing
      start, stop, step = key.indices(len(self))
      sliced = deepcopy(self)
      sliced_data = self.dataframe.iloc[start:stop:step]
      sliced.dataframe = sliced_data
      return  sliced

    else: #If given a single index 
      if torch.is_tensor(key):
        key = key.tolist()
      
      #For one sample Dataset[i]:
      dataframe = self.dataframe

      label = dataframe.iloc[key, 2] #retrieveing the label
      dataset = dataframe.iloc[key,0] #retrieveing the dataset-name
      T = dataframe.iloc[key,1] #retrieveing the timeseries-length
      time_series_with_nan = dataframe.iloc[key,3:].to_numpy() #retrieveing the timeseries (containing nan-values at the end)
      time_series = time_series_with_nan[:T] #Removing nan_values at the end

      sample = {'time_series': time_series, 'label': label, 'dataset': dataset, "T": T} #a sample is one timeseries with it's corresponding label (and som xtra information)

      if self.transform: #transform sample
        sample = self.transform(sample)

      return sample
  
  def info(self):#Function to print information about the dataset
    print("Datasets included: ", self.Datasets)
    print("Number of classes : ", self.classes)
    print("Size of dataset: ", len(self))
  
  def shuffle(self):
    self.dataframe = self.dataframe.sample(frac = 1)
  

### Transformation class; segment timeseries into two augmentations
class TwoSegments(object):
  def __init__(self, horizon=0.3, window_gap=1, random_startpos = False, random_horizon=False, random_window_gap=False):
    #horizon: horizon*T = window_length; the length of the two augmentations
    #window_gap: the gap bewteen the two augmentations
    #random_startpos: True means that the first augmentation starts at a random position
    #random_horizon: True means that a random horizon is chosen
    #random_window_gap: True means that a random window_gap is chosen

    self.horizon = horizon
    self.window_gap = window_gap
    self.random_startpos = random_startpos
    self.random_horizon = random_horizon
    self.random_window_gap = random_window_gap
        
  def __call__(self, sample):

    dataset = sample['dataset']
    time_series = sample['time_series']
    T = sample['T']
    label = sample['label']

    #horizon
    if (self.random_horizon==True):
      possible_horizons = [0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
      horizon = random.choice(possible_horizons) #draw a random horizon
    else:
      horizon = self.horizon

    #window gap
    if (self.random_window_gap==True):
      possible_window_gaps =[0,1,2,3,4,5,6,7,8,9,10]
      window_gap = random.choice(possible_window_gaps) #draw random window_gap
    else:
      window_gap = self.window_gap

    window_length = int(horizon*T) #length of each augmentation
    #window_length = 20

    #finding start-position of the first augmentation
    if (self.random_startpos == True): #if random start_position
      max_possible_startposition = T-(2*window_length+window_gap) #the maximal start-position of the first augmentation
      possible_startpossisions = [i for i in range(max_possible_startposition_aug1)] #The possible start positions of the first augmentation
      start_pos = random.choice(possible_startpossisions) #draw a random startposition
    else:
      start_pos = 0 

    #make the two augmentations of the timeseries
    augmentation_1 = time_series[start_pos : (start_pos+window_length)]
    augmentation_2 = time_series[(start_pos+window_length+window_gap) : (start_pos+window_length+window_gap)+window_length]

    #create a new sample with the two augmentations
    #new_sample = {'time_series': time_series, 'aug1': augmentation_1, 'aug2': augmentation_2, 'label': label, 'dataset': dataset, "T": T}
    new_sample = {'aug1': augmentation_1, 'aug2': augmentation_2, 'label': label, 'dataset': dataset, "T": T}

    return new_sample

### Transformation class; convert into Tensor-data for PyTorch-use
class ToTensor(object):
  def __call__(self, sample):
    dataset = sample['dataset']
    T = sample['T']
    label = sample['label']

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #if (len(sample)== 6):
    if (len(sample)== 5):
      #time_series = sample['time_series'].astype(float)
      #time_series = time_series[np.newaxis,:] #tensor

      aug1 = sample['aug1'].astype(float)
      aug1 = aug1[np.newaxis,:] #tensor

      aug2 = sample['aug2'].astype(float)
      aug2 = aug2[np.newaxis,:] # tensor
      
      label = label.astype(float)

      #time_series_tensor = torch.tensor(time_series, dtype=torch.float32, device=device)
      aug1_tensor = torch.tensor(aug1, dtype=torch.float32, device=device)
      aug2_tensor = torch.tensor(aug2, dtype=torch.float32, device=device)
      label_tensor = torch.tensor(label, dtype=torch.long, device=device)

      #torch_sample = {'time_series': time_series_tensor, 'aug1': aug1_tensor, 'aug2': aug2_tensor, 'label': label_tensor, 'dataset': dataset, "T": T}
      torch_sample = { 'aug1': aug1_tensor, 'aug2': aug2_tensor, 'label': label_tensor, 'dataset': dataset, "T": T}


    else:
      time_series = sample['time_series'].astype(float)
      label = label.astype(float)

      time_series_tensor = torch.tensor(time_series, dtype=torch.float32, device=device)
      label_tensor = torch.tensor(label, dtype=torch.long, device=device)

      torch_sample = {'time_series': time_series_tensor, 'label': label_tensor, 'dataset': dataset, "T": T}

    return torch_sample

#Function to split into train and test dataset given a Timeseries_Dataset object
def data_split(dat, train_size=0.8):
  N = len(dat)
  n = int(train_size*N)

  dat.shuffle()
  train = dat[:n]
  test = dat[n:]

  return train, test



In [41]:
##### MODELS #######

#FCN modellen fra prosjektoppgave
class FCN(nn.Module):
  def __init__(self):#, classes):
    super(FCN, self).__init__()
    self.conv1 = nn.Conv1d(1, 128, 9, padding=(9 // 2))
    self.bnorm1 = nn.BatchNorm1d(128)        
    self.conv2 = nn.Conv1d(128, 256, 5, padding=(5 // 2))
    self.bnorm2 = nn.BatchNorm1d(256)
    self.conv3 = nn.Conv1d(256, 128, 3, padding=(3 // 2))
    self.bnorm3 = nn.BatchNorm1d(128)        
    #self.classification_head = nn.Linear(128, classes)
    self.output_dim = 128

  def forward(self, x):
    b1_class = F.relu(self.bnorm1(self.conv1(x)))
    b2_class = F.relu(self.bnorm2(self.conv2(b1_class)))
    b3_class = F.relu(self.bnorm3(self.conv3(b2_class)))

    features_class = torch.mean(b3_class, 2) 
    #out_class = self.classification_head(features_class)

    return features_class

In [42]:
### SIMPLE SIAMESE REPRESENTATION LEANRING MODELLEN ###

## DISTANCE FUNCTION ##
#The distance to minimize  (Negative Cosine similarity)
def D(p, z):
  z = z.detach() #stop gradient
  p = p
  return - F.cosine_similarity(p, z, dim=1).mean()
  #return F.cosine_similarity(p, z, dim=1).mean()


## PROJECTION MLP ## ( in f(x))
#has 3 FC layers with BN, the output FC has no ReLU, hidden FC is 2048-d
class projection_MLP(nn.Module):
  def __init__(self, in_dim, hidden_dim=2048, out_dim=2048): 
    super().__init__()
    #Layer 1
    self.FC1 = nn.Sequential(
      nn.Linear(in_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True)  )
    #Layer 2
    self.FC2 = nn.Sequential(
      nn.Linear(hidden_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True)  )
    #Layer 3 (output)
    self.FC3 = nn.Sequential(
      nn.Linear(hidden_dim, out_dim),
      nn.BatchNorm1d(hidden_dim) )
  
  def forward(self, x):
    x1 = self.FC1(x)
    x2 = self.FC2(x1)
    x_out = self.FC3(x2)
    return x_out


## PREDICTION MLP ## h(z)
#has 2 FC layers, only BN and ReLU on hidden layer (explained why in SimSiam paper). 
# input: z = f(x) dim = 2048
# output: p = h(z) dim = 2048
class prediction_MLP(nn.Module):
  def __init__(self, in_dim=2048, hidden_dim=512, out_dim=2048): # bottleneck structure
    super().__init__()
    #Layer 1
    self.FC1 = nn.Sequential(
      nn.Linear(in_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True) )
    #Layer 2 (output)
    self.FC2 = nn.Linear(hidden_dim, out_dim)
  
  def forward(self, x):
    x1 = self.FC1(x)
    x_out = self.FC2(x1)
    return x_out


## SIMSIAM MODEL ##
class SimSiam(nn.Module):
  def __init__(self, backbone=FCN()): #take backbone as input
    super().__init__()
    self.backbone = backbone
    self.projector = projection_MLP(backbone.output_dim) 
    
    #Encoder; z = f(x)
    self.encoder = nn.Sequential( 
      self.backbone,
      self.projector )

    #Predictor; p = h(z)
    self.predictor = prediction_MLP()
  
  def forward(self, x1, x2):
    #x1, x2: augmentations of x
    f = self.encoder
    h = self.predictor

    z1, z2 = f(x1), f(x2)
    p1, p2 = h(z1), h(z2)

    #Symmetric loss
    Loss = D(p1,z2)/2 + D(p2,z1)/2 
    return Loss


In [47]:

#Importing Dataset class from other ipynb file
#!pip install import_ipynb
#drive.mount('/content/drive')
#%cd '/content/drive/MyDrive/Colab Notebooks'

#!ls
#import import_ipynb
#from Dataset_class import *

Dataset_small = Datasets = ["ChlorineConcentration"]
Small_dataset = Timeseries_Dataset(Dataset_small, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
train_small, test_small = data_split(Small_dataset)

#Dataset_big = ["Big_dataset"]
#Big_dataset = Timeseries_Dataset(Dataset_big, Drive=True, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
#train_big, test_big = data_split(Big_dataset)


Loading ' ChlorineConcentration ' from github


In [ ]:


epochs = 10
train_bs = 512
momentum = 0.9
weight_decay =0.0001
warmup_epochs = 0 #=10 for for batch sizes  > 1024
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_lr = 0.05
lr = (base_lr*train_bs)/256
#lr= 0.0001

model = SimSiam().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train_loader = DataLoader(train_small, batch_size = train_bs, shuffle=True)
losses =[]

for e in range(epochs):
  model.train()
  for batch in enumerate(train_loader):
    x1_batch = batch[1]['aug1']
    x2_batch = batch[1]['aug2']

    model.zero_grad()
    loss = model.forward(x1_batch, x2_batch).mean()
    #loss = model.forward(x1_batch.to(device,non_blocking=True), x2_batch.to(device,non_blocking=True)).mean()
    loss.backward()
    optimizer.step()

    losses.append(loss.item())
  
  avg_loss = np.mean(losses)
  print("e: ",e," Avg Loss: ", avg_loss)







